## Feature Extaction Test

In [1]:
import sys
sys.path.insert(0, '..')
from metrics.metrics import get_tp_fp_fn, quick_circle_comparison, circle_comparison
from general_utils.plots import plot_bboxes_over_image, simple_im_show, simple_im_show2
from mc_candidate_proposal.hough_mc import HoughCalcificationDetection
from feature_extraction.feature_extraction import CandidatesFeatureExtraction
from database.dataset import *
import feature_extraction.haar_features.haar_modules as hm

import pickle

### Database initialization

In [2]:
db = INBreast_Dataset(
        return_lesions_mask=True,
        level='image',
        max_lesion_diam_mm=1.0,
        partitions=['train', 'validation'],
        extract_patches=False,
        min_breast_fraction_roi=0.5,
        normalize=None,
        n_jobs=-1,
)

### Detectors Parameters and initialization

In [3]:
hd = HoughCalcificationDetection()
CENTER_CROP_PATCH = 14
PATCH_SIZE = 14

GABOR_PARAMS = {'scale': 2, 'orientation': 3,
                'max_freq': 0.2, 'ksize': (20, 20), 'sigma': 3}
WAVELET_PARAMS = {'angles': [0, np.pi/4, np.pi/2]}
HAAR_PARAMS = {
    'skimage': {
        'feature_type': [], 'feature_coord': []
    },
    'ours': {
        'horizontal_feature_types': [(hm.Feature3h3v, 3, 3)],
        'rotated_feature_types': None,
        'horizontal_feature_selection': None,
        'rotated_feature_selection': None,
    },
    'patch_size': PATCH_SIZE
}


# IS THIS FINAL HAAR? MAYBE
path = (Path.cwd().parent / f'feature_extraction/haar_features/')
with open(path/'final_feat_selection.p', 'rb') as f:
    selection = pickle.load(f)
    
HAAR_PARAMS = {
    'skimage': {
        'feature_type': selection['skimage_haar_feature_types_sel'],
        'feature_coord': selection['skimage_haar_feature_coords_sel']
    },
    'ours': {
        'horizontal_feature_selection': selection['hor_feats_selection'].tolist(),
        'rotated_feature_selection': selection['rot_feats_selection'].tolist(),
        'rotated_feature_types': None,
        'horizontal_feature_types': None
    },
    'patch_size': 14
}

cfe = CandidatesFeatureExtraction(patch_size=PATCH_SIZE, fos=True, gabor_params=GABOR_PARAMS,
                                  wavelet_params=WAVELET_PARAMS, haar_params=HAAR_PARAMS)

### Detector initialization

### FE

In [4]:
fdf = []
fns_df = []
for idx in tqdm(range(len(db))):
    # extracting data
    db_sample = db[idx]
    image = db_sample['img']
    image_id = db.df.iloc[idx].img_id
    image_mask = db_sample['lesion_mask']

    # candidate selection
    candidates = hd.detect(image, image_id, load_processed_images=True, hough2=False)[0]

     # labeling of candidates:
    tp, fp, fn = utils.get_tp_fp_fn_center_patch_criteria(
        candidates, image_mask, None, PATCH_SIZE)
    
    candidates = tp + fp
    labels = np.array([True] * len(tp) + [False] * len(fp))
    
    # feature extraction
    X = cfe.extract_features(candidates, image)
    res = pd.DataFrame(X, columns=cfe.feature_names)
    
    res['img_id'] = image_id
    res['labels'] = labels

    
    # Generate a fn dataframe to compute frocs
    fn_df = pd.DataFrame(fn, columns=['x', 'y', 'radius'])
    fn_df['labels'] = 'FN'
    fn_df['img_id'] = image_id
    fns_df.append(fn_df)
    fdf.append(res)

all_data_df = pd.concat(fdf, ignore_index=True)
fns_df = pd.concat(fns_df, ignore_index=True)

100%|██████████| 154/154 [3:14:59<00:00, 75.97s/it]   


In [5]:
pd.to_pickle((all_data_df, fns_df), 'feature_sets/hough1_all_feat_1pxsize.pkl')

In [6]:
fdf = []
fns_df = []
for idx in tqdm(range(len(db))):
    # extracting data
    db_sample = db[idx]
    image = db_sample['img']
    image_id = db.df.iloc[idx].img_id
    image_mask = db_sample['lesion_mask']

    # candidate selection
    candidates = hd.detect(image, image_id, load_processed_images=True, hough2=True)[1]

     # labeling of candidates:
    tp, fp, fn = utils.get_tp_fp_fn_center_patch_criteria(
        candidates, image_mask, None, PATCH_SIZE)
    
    candidates = tp + fp
    labels = np.array([True] * len(tp) + [False] * len(fp))
    
    # feature extraction
    X = cfe.extract_features(candidates, image)
    res = pd.DataFrame(X, columns=cfe.feature_names)
    
    res['img_id'] = image_id
    res['labels'] = labels

    
    # Generate a fn dataframe to compute frocs
    fn_df = pd.DataFrame(fn, columns=['x', 'y', 'radius'])
    fn_df['labels'] = 'FN'
    fn_df['img_id'] = image_id
    fns_df.append(fn_df)
    fdf.append(res)

all_data_df = pd.concat(fdf, ignore_index=True)
fns_df = pd.concat(fns_df, ignore_index=True)

100%|██████████| 154/154 [46:18<00:00, 18.04s/it]


In [ ]:
pd.to_pickle((all_data_df, fns_df), 'feature_sets/hough2_all_feat_1pxsize.pkl')